In [1]:
import numpy as np
import pickle as pkl
import os
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from math import floor, ceil
import time

In [2]:
def get_max_and_concat(*args):
    size = 0
    for arg in args:
        size += arg.shape[-1]
    final = np.zeros(size)
    
    arg_size = 0

    for arg in args:
        for i in range(arg.shape[-1]):
            final[i+arg_size] = np.max(arg[:,:,i])
        arg_size += arg.shape[-1]

    return final



In [3]:
def max_pooled(previous_features, features, label):
    box_prev = previous_features[
        floor(label[0]*previous_features.shape[0]):
        ceil(label[2]*previous_features.shape[0]), 
        floor(label[1]*previous_features.shape[1]):
        ceil(label[3]*previous_features.shape[1]),:]
    box_cur = features[
        floor(label[0]*features.shape[0]):
        ceil(label[2]*features.shape[0]), 
        floor(label[1]*features.shape[1]):
        ceil(label[3]*features.shape[1]),:]

    return get_max_and_concat(box_prev, previous_features, box_cur, features)

In [4]:
SESSIONS_PATH = "/home/wc-gpu/storage4tb/session_data_thesis/sessions160000_165000"
IMAGE_SIZE = (30, 20)
plot_features_and_annotations = True

In [5]:
classes = ["same", "add", "remove", "move_from", "move_to"]

In [6]:
total_images = 0
sessions = os.listdir(SESSIONS_PATH)
actual_sessions = []

for session in sorted(sessions, reverse=True):
    try:
        a = int(session)
        session_path = os.path.join(SESSIONS_PATH, session)
        boxes_path = os.path.join(session_path, "boxes")
        features_path = os.path.join(session_path, "features")
        
        if os.path.exists(features_path) and os.path.exists(boxes_path):
            actual_sessions.append(session)
            images = [f for f in os.listdir(session_path) if f.endswith(".jpg")]
            total_images += len(images)
    except:
        print (session)
        continue

old_ds
max_breaks.pkl


In [7]:
X = []
Y = []

progress = 0

processed = 0
time_avg = 0

for session in sorted(actual_sessions):
    try:
        features_path = os.path.join(SESSIONS_PATH, session, "features")
        boxes_path = os.path.join(SESSIONS_PATH, session, "boxes")
        session_path = os.path.join(SESSIONS_PATH, session)
        previous_features = None
        previous_boxes = None
        for image_file in sorted(os.listdir(session_path)):
            try:
                if image_file.endswith(".jpg"):

                    t0 = time.time()

                    feature_path = os.path.join(features_path, image_file + ".feature.prod.new.better")
                    if not os.path.exists(features_path):
                        break
                    boxe_path = os.path.join(boxes_path, image_file + ".box.better")
                    features = None
                    boxes = None
                    with open(feature_path, "rb") as f:
                        features = np.load(f)[0]
                    with open(boxe_path, "rb") as f:
                        boxes = pkl.load(f)

                    if previous_features is not None:
                        for label in boxes[0]:
                            max_pool = max_pooled(previous_features, features, label)
                            X.append(max_pool)
                            Y.append(0)

                        for label in boxes[1]:
                            forward = max_pooled(previous_features, features, label)
                            backward = max_pooled(features, previous_features, label)
                            if(label[4] == "add"):
                                X.append(forward)
                                Y.append(1)
                                X.append(backward)
                                Y.append(2)
                            elif("move" in label[4] and "move_from" not in label[4]):
                                X.append(forward)
                                Y.append(4)
                                X.append(backward)
                                Y.append(3)

                        for label in previous_boxes[1]:
                            forward = max_pooled(previous_features, features, label)
                            backward = max_pooled(features, previous_features, label)
                            if(label[4] == "remove"):
                                X.append(forward)
                                Y.append(2)
                                X.append(backward)
                                Y.append(1)
                            elif("move_from" in label[4]):
                                X.append(forward)
                                Y.append(3)
                                X.append(backward)
                                Y.append(4)

                    previous_features = features
                    previous_boxes = boxes

                    total_time = time_avg * processed
                    processed += 1

                    t1 = time.time()

                    total = t1-t0
                    time_avg = (total_time + total) / processed

                    print ("progress", session, "  : ", progress / total_images, progress, total_images,
                           "   |   time", total, "s,   total passed: ", time_avg*processed / 3600 ," h  left: ", time_avg* (total_images - progress) / 3600, " h", end="\r")

                    progress += 1
                
            except Exception as ex:
                print (session, ex)
    except Exception as ex:
        print (session, ex)
    

In [8]:
X = np.asarray(X)
Y = np.asarray(Y)

In [9]:
import pickle as pkl

with open(SESSIONS_PATH + "/dataset_prod_new_model_multiclass.pkl", "wb") as f:
    pkl.dump([X, Y], f)

In [10]:
Y.shape


(68208,)

In [ ]:
# for session in sorted(os.listdir(SESSIONS_PATH)):
#     try:
#         a = int(session)

#         features_path = os.path.join(SESSIONS_PATH, session, "features")
#         boxes_path = os.path.join(SESSIONS_PATH, session, "boxes")
#         session_path = os.path.join(SESSIONS_PATH, session)
#         if not os.path.exists(os.path.join(features_path)) or not os.path.exists(os.path.join(boxes_path)) :
#             continue

#         for image_file in sorted(os.listdir(session_path)):
#             try:
#                 if not image_file.endswith(".jpg"):
#                     continue
#                 feature_path = os.path.join(features_path, image_file + ".feature.prod.new")
                
#                 boxe_path = os.path.join(boxes_path, image_file + ".box")
#                 features = None
#                 boxes = None
#                 with open(feature_path, "rb") as f:
#                     features = np.load(f)
#                 with open(boxe_path, "rb") as f:
#                     boxes = pkl.load(f)
                
#                 features = features[0]
#                 print ("SHAPE", features.shape)
#                 image_np = features[:,:,500]
#                 image_np *= 255.0/image_np.max()  

#                 fig,ax = plt.subplots(1, figsize=IMAGE_SIZE)

#                 ax.imshow(image_np)
#                 for label in boxes[0]:
#                     rect = patches.Rectangle(
#                         (label[1]*image_np.shape[1],label[0]*image_np.shape[0]),
#                         (label[3]-label[1])*image_np.shape[1],
#                         (label[2]-label[0])*image_np.shape[0],
#                         linewidth=5,edgecolor='r',facecolor='none')
#                     ax.add_patch(rect)

#                 for label in boxes[1]:
#                     rect = patches.Rectangle(
#                         (label[1]*image_np.shape[1],label[0]*image_np.shape[0]),
#                         (label[3]-label[1])*image_np.shape[1],
#                         (label[2]-label[0])*image_np.shape[0],
#                         linewidth=5,edgecolor='g',facecolor='none')
#                     ax.add_patch(rect)
#                 plt.show(IMAGE_SIZE)
                
                
#                 print("negatives")
#                 for label in boxes[0]:
# #                     print (
# #                         floor(label[0]*image_np.shape[0]),
# #                         ceil(label[2]*image_np.shape[0]),
# #                         floor(label[1]*image_np.shape[1]),
# #                         ceil(label[3]*image_np.shape[1]))
#                     rect = features[
#                         floor(label[0]*image_np.shape[0]):
#                         ceil(label[2]*image_np.shape[0]), 
#                         floor(label[1]*image_np.shape[1]):
#                         ceil(label[3]*image_np.shape[1])]
#                     fig,ax = plt.subplots(1)
#                     ax.imshow(rect[:,:,500])
#                     plt.show()
                    
                
                
#                 print("positives")
#                 for label in boxes[1]:
#                     rect = features[
#                         floor(label[0]*image_np.shape[0]):
#                         ceil(label[2]*image_np.shape[0]), 
#                         floor(label[1]*image_np.shape[1]):
#                         ceil(label[3]*image_np.shape[1])]
#                     fig,ax = plt.subplots(1)
#                     ax.imshow(rect[:,:,500])
#                     plt.show()
                    
                
#             except Exception as ex:
#                 print (session, ex)
#                 continue
#         break
#     except Exception as ex:
#         print (session, ex)
#         continue
    